In [1]:
import xyzpy as xyz
import quimb as qu
import quimb.tensor as qtn
from quimb.experimental.operatorbuilder import *

In [2]:
Lx = 4
Ly = 3
N = Lx * Ly
num_up = 6
num_down = 6
subspace = ((N, num_up), (N, num_down))

In [3]:
edges = qtn.edges_2d_square(Lx, Ly)
H = fermi_hubbard_from_edges(edges)
H.show()

SparseOperatorBuilder(nsites=24, nterms=80, locality=4))
+-...................... -1.0
-+...................... -1.0
............+-.......... -1.0
............-+.......... -1.0
+zz-.................... -1.0
-zz+.................... -1.0
............+zz-........ -1.0
............-zz+........ -1.0
.+-..................... -1.0
.-+..................... -1.0
.............+-......... -1.0
.............-+......... -1.0
.+zz-................... -1.0
.-zz+................... -1.0
.............+zz-....... -1.0
.............-zz+....... -1.0
..+zz-.................. -1.0
..-zz+.................. -1.0
..............+zz-...... -1.0
..............-zz+...... -1.0
...+-................... -1.0
...-+................... -1.0
...............+-....... -1.0
...............-+....... -1.0
...+zz-................. -1.0
...-zz+................. -1.0
...............+zz-..... -1.0
...............-zz+..... -1.0
....+-.................. -1.0
....-+.................. -1.0
................+-...... -1.0
.............

# build a sparse matrix to find groundstate of

In [4]:
%%time
S = H.build_sparse_matrix(*subspace, parallel=True)
S

CPU times: user 10.4 s, sys: 625 ms, total: 11 s
Wall time: 11.3 s


<853776x853776 sparse matrix of type '<class 'numpy.float64'>'
	with 16686516 stored elements in Compressed Sparse Row format>

In [5]:
%%time
ge, gs = qu.eigh(S, k=1, which='SA')
ge

CPU times: user 27.7 s, sys: 27.5 s, total: 55.1 s
Wall time: 8.18 s


array([-13.81834891])

# construct and find the expectation value of a different operator

In [6]:
hs = H.hilbert_space
hs.get_size(*subspace)

853776

In [7]:
C = SparseOperatorBuilder(
    terms=[
        (1.0, '+-', ('↑', 1, 1), ('↑', 2, 1)),
    ], 
    # need to embed into larger Hilbert space of Hamiltonian
    hilbert_space=hs
)

# ensure fermionic
C.jordan_wigner_transform()

In [8]:
C.show()

SparseOperatorBuilder(nsites=24, nterms=1, locality=4))
....+zz-................ +1.0


In [9]:
# build sparse matrix
CS = C.build_sparse_matrix(*subspace, parallel=True)
CS

<853776x853776 sparse matrix of type '<class 'numpy.float64'>'
	with 232848 stored elements in Compressed Sparse Row format>

In [10]:
# find the groundstate expectation
qu.expec(CS, gs)

0.3473426081278192

# VMC usage

For VMC we just need to compute coupling between computation basis
configurations (i.e. a single row of the sparse matrix).

In [11]:
config = hs.rand_config()
config

{('↑', 0, 0): 1,
 ('↑', 0, 1): 1,
 ('↑', 0, 2): 0,
 ('↑', 1, 0): 1,
 ('↑', 1, 1): 0,
 ('↑', 1, 2): 1,
 ('↑', 2, 0): 1,
 ('↑', 2, 1): 0,
 ('↑', 2, 2): 0,
 ('↑', 3, 0): 1,
 ('↑', 3, 1): 1,
 ('↑', 3, 2): 1,
 ('↓', 0, 0): 0,
 ('↓', 0, 1): 1,
 ('↓', 0, 2): 1,
 ('↓', 1, 0): 0,
 ('↓', 1, 1): 0,
 ('↓', 1, 2): 1,
 ('↓', 2, 0): 1,
 ('↓', 2, 1): 1,
 ('↓', 2, 2): 1,
 ('↓', 3, 0): 0,
 ('↓', 3, 1): 0,
 ('↓', 3, 2): 0}

In [12]:
configs, coeffs = H.config_coupling(config)

In [21]:
print(configs[:2], '...')
coeffs

[{('↑', 0, 0): 1, ('↑', 0, 1): 1, ('↑', 0, 2): 0, ('↑', 1, 0): 1, ('↑', 1, 1): 0, ('↑', 1, 2): 1, ('↑', 2, 0): 1, ('↑', 2, 1): 0, ('↑', 2, 2): 0, ('↑', 3, 0): 1, ('↑', 3, 1): 1, ('↑', 3, 2): 1, ('↓', 0, 0): 1, ('↓', 0, 1): 0, ('↓', 0, 2): 1, ('↓', 1, 0): 0, ('↓', 1, 1): 0, ('↓', 1, 2): 1, ('↓', 2, 0): 1, ('↓', 2, 1): 1, ('↓', 2, 2): 1, ('↓', 3, 0): 0, ('↓', 3, 1): 0, ('↓', 3, 2): 0}, {('↑', 0, 0): 1, ('↑', 0, 1): 0, ('↑', 0, 2): 1, ('↑', 1, 0): 1, ('↑', 1, 1): 0, ('↑', 1, 2): 1, ('↑', 2, 0): 1, ('↑', 2, 1): 0, ('↑', 2, 2): 0, ('↑', 3, 0): 1, ('↑', 3, 1): 1, ('↑', 3, 2): 1, ('↓', 0, 0): 0, ('↓', 0, 1): 1, ('↓', 0, 2): 1, ('↓', 1, 0): 0, ('↓', 1, 1): 0, ('↓', 1, 2): 1, ('↓', 2, 0): 1, ('↓', 2, 1): 1, ('↓', 2, 2): 1, ('↓', 3, 0): 0, ('↓', 3, 1): 0, ('↓', 3, 2): 0}] ...


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       3.])